Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось.

## 1. Загрузка данных

In [92]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score


In [93]:
clients = pd.read_csv('/datasets/insurance.csv')

In [94]:
clients.head(10)

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0
5,1,43.0,41000.0,2,1
6,1,39.0,39700.0,2,0
7,1,25.0,38600.0,4,0
8,1,36.0,49700.0,1,0
9,1,32.0,51700.0,1,0


In [95]:
clients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [96]:
clients.isnull().sum()

Пол                  0
Возраст              0
Зарплата             0
Члены семьи          0
Страховые выплаты    0
dtype: int64

Загрузил данные, посмотрел info, проверил на пропуски (хотя и по info видно что их нет)

## 2. Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

### Вычисления:



$$
w = ((XP)^T XP)^{-1} (XP)^T y = (P^T * X^T * X * P)^{-1} * (XP)^T y =
$$

$$
= P^{-1} * (X^TX)^{-1} * (P^T)^{-1} * P^T * X^T * y =
$$

$$
= P^{-1} * (X^TX)^{-1} * X^T * y 
$$


$$
a = X*P*P^{-1} * (X^TX)^{-1} * X^T * y   =  X(X^T)^{-1}X^{-1}X^Ty = X(X^TX)^{-1}X^Ty
$$

**Ответ:** Качество линейной регрессии не изменится.

**Обоснование:** при раскрытии скобок в формуле обучении линейной регрессии я получил $(P^T)^{-1}$ и $ P^T $ которые образуют единичную квадратную матрицу, которую можно сократить.
 
 Далее, это формулу я подставил в формулу предсказания, где получил $P^{-1}$ и $ P $ которые так же сокращаются.
 
 В результате у меня осталась исходная формула.

## 3. Алгоритм преобразования

### **Алгоритм**

##### Признаки features я умножу на рандомную квадратную обратимую матрицу. Для дальнейшей расшифровки данных зашифрованные данные надо удет умножить на обратную матрицу

### **Обоснование**

##### При умножении на матрицу мы получаем данные, понять которые без обратного преобразования невозможно. И как я выяснил по формулам выше, на качество модели такиое преобразование не отразится

In [100]:
target = clients['Страховые выплаты']
features = clients.drop(['Страховые выплаты'], axis=1)


In [101]:
train_features, valid_features, train_target, valid_target =  train_test_split(features, target, test_size = 0.25, random_state = 12345)

создал таргет и фитчи, разбил выборку на valid и train 

In [102]:
model = LinearRegression()
model.fit(train_features, train_target)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [103]:
predicted = model.predict(valid_features)
r2 = r2_score(valid_target, predicted)
r2

0.435227571270266

обучил линейную регрессию на обучающей выборке, сделал предсказание на валидационной, получил метрику r2 0.43



## 4. Проверка алгоритма

In [104]:
#random_marix_1 = np.random.rand(4,4)
random_marix = [[0.250796  , 0.84617143, 0.02477692, 0.5175202 ],
       [0.57730995, 0.23139768, 0.85821472, 0.54718381],
       [0.61037571, 0.38988881, 0.83823909, 0.61311635],
       [0.95072697, 0.74551835, 0.20033647, 0.10551149]]

Создал рандомную квадратную матрицу 4х4, кодом ниже проверил ее на обратимость и зафиксировал матрицу в переменную random_marix

In [105]:
random_marix_inv = np.linalg.inv(random_marix)
random_marix_inv 


array([[  5.92896911,  35.91995639, -37.34619446,   1.65273527],
       [ -6.66273951, -41.05573345,  42.35099705,  -0.50187705],
       [ -8.80655536, -45.0029624 ,  47.77249543,  -1.02012883],
       [ 10.37458518,  51.87554437, -53.4347893 ,   0.06850028]])

сделал обратную матрицу

In [106]:
coded_features_matrix = features.values @ random_marix
coded_features_matrix

array([[30299.50644692, 19349.56397066, 41612.07078091, 30433.6285279 ],
       [23221.78396467, 14827.16459163, 31892.76363359, 23323.69726675],
       [12834.63189855,  8194.37554272, 17627.90911688, 12891.31168049],
       ...,
       [20705.18422194, 13223.3496493 , 28433.87011834, 20795.79896418],
       [19975.08951281, 12757.53756244, 27429.92475317, 20061.77674349],
       [24798.62002757, 15837.55651082, 34056.76217955, 24908.46798837]])

умножил признаки features переведенные в матрицу методом (values) на мою рандомную матрицу, получили зашифрованные данные

In [107]:
pd.DataFrame(coded_features_matrix @ random_marix_inv, columns = features.columns).round().astype(int)


,Пол,Возраст,Зарплата,Члены семьи
0,1,41,49600,1
1,0,46,38000,1
2,0,29,21000,0
3,0,21,41700,2
4,1,28,26100,0
...,...,...,...,...
4995,0,28,35700,2
4996,0,34,52400,1
4997,0,20,33900,2
4998,1,22,32700,3


расшифорвываю данные, умножая закодированные на обратную матрицу. Из полученной матрицы я делаю data frame,
округляю данные и приводя их к целым числам (метод round и astype(int)) 


### Проверка качества модели

In [108]:
train_features1, valid_features1, train_target1, valid_target1 =  train_test_split(coded_features, target, test_size = 0.25, random_state = 12345)

Так же как с исходными данные, делю данные на выборки, однако в качестве фитчей беру закодированные данные.

In [109]:
model1 = LinearRegression()
model1.fit(train_features1, train_target1)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [110]:
predicted1 = model1.predict(valid_features1)
r2_coded = r2_score(valid_target1, predicted1)
r2_coded

0.4352275712937975

In [111]:
r2

0.435227571270266

Обучил модель, и проверил метрику R2, она оказалась точно такой же как на модели с исходными данными - (0.43)

### Вывод

#### В данном проекте я опробвоал на практике метод шифрования данных с помощью умножения их на квадратную матрицу, и посмотрел как данный метод отразится на моделях машинного обучения.

#### Практически (построив модель и проверив метрики) и теоретически (на формулах) мне удалось доказать что качество модели линейной регресии не изменилось.

#### Данный метод хорошо подойдет шифрования данных, которые будут использоваться в дальнейшем в машинном обучении 